In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample-resume2/RESUME (Hss).pdf
/kaggle/input/testing101/testresume.pdf
/kaggle/input/sample-resume3/External_Apping_resume.pdf
/kaggle/input/doc-format/RESUME (Hss).docx
/kaggle/input/sample-resume/Devashish Bhave_Resume.pdf


# Preparation

First lets take input 

In [2]:
!pip install pymupdf
!pip install PyPDF2
!pip install fpdf
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 30.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 51.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40703 sha256=bee576c7cb09f03ebf6e506ece18046575a53d0e801ee7b22827953781a4557d
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.2 MB/s eta 0:00:00a 0:00:01


In [3]:
import fitz  # PyMuPDF
import re
import csv
import docx

# Extraction function

In [4]:
def extract_details(text):
    details = {
        "Candidate Name": "",
        "Phone Number": "",
        "Email Address": ""
    }

    # Extract candidate name 
    name_match =name_match = re.search(r"([A-Z][a-z]+[\s-]+[A-Z][a-z.]+[\s-]+[A-Z][a-z.])+", text)
    if name_match:
        details["Candidate Name"] = name_match.group(0)

    # Extract phone number 
    phone_match = re.search(r'''(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)
                    [-\.\s]*\d{3}[-\.\s]??\d{4}|\d{5}[-\.\s]??\d{4})''', text)
    if phone_match:
        details["Phone Number"] = phone_match.group(0)

    # Extract email address 
    if not details["Email Address"]:
        email_match = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
        if email_match:
            details["Email Address"] = email_match.group(0)

    return details

In [5]:
def extract_contact_info_from_pdf(pdf_file_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_file_path)

    # Initialize variables
    contact_info = {
        "Candidate Name": "",
        "Phone Number": "",
        "Email Address": ""
    }

    # Loop each page 
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text()
        details = extract_details(text)

        # Update contact_info 
        for key in details:
            if details[key]:
                contact_info[key] = details[key]

    # Close the PDF file
    pdf_document.close()

    return contact_info

In [6]:
def extract_contact_info_from_docx(docx_file_path):
    # Open the DOCX file
    doc = docx.Document(docx_file_path)

    # Initialize variables 
    contact_info = {
        "Candidate Name": "",
        "Phone Number": "",
        "Email Address": ""
    }

    # Loop each paragraph 
    for para in doc.paragraphs:
        text = para.text
        details = extract_details(text)

        # Update contact_info 
        for key in details:
            if details[key]:
                contact_info[key] = details[key]

    return contact_info

# Checking a whether a supported format is used or not

In [7]:
def extract_contact_info_from_file(file_path):
    if file_path.lower().endswith('.pdf'):
        return extract_contact_info_from_pdf(file_path)
    elif file_path.lower().endswith('.docx'):
        return extract_contact_info_from_docx(file_path)
    else:
        raise ValueError("Unsupported file format. Only PDF and DOCX are supported.")


Saving details

In [8]:
def save_contact_info_to_csv(contact_info, output_csv_file):
    with open(output_csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=["Candidate Name", "Phone Number", "Email Address"])
        writer.writeheader()
        writer.writerow(contact_info)

# Using on text resume

In [21]:
file_path = "/kaggle/input/testing-102/RESUME Test 02.pdf"
output_csv_file = "Contact_information.csv"
contact_info = extract_contact_info_from_file(file_path)
save_contact_info_to_csv(contact_info, output_csv_file)